# 3.Preprocessing and Training

### Import Libaries

In [1]:
import pandas as pd
import numpy as np
import os
import pickle
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import __version__ as sklearn_version
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler,OneHotEncoder
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression

### 1.Load Data

In [2]:
datafilepath = "../data/interim/top_10_crime_data.csv"
df_top10_crime = pd.read_csv(datafilepath,index_col =0)

In [3]:
df_top10_crime.head(3)

,TIME_OCC,AREA,AREA_NAME,RPT_DIST_NO,PART_1-2,CRM_CD,CRM_CD_DESC,MOCODES,VICT_AGE,VICT_SEX,...,latitude,longitude,MTH_RPTD,DAY_RPTD,YEAR_RPTD,MTH_OCC,DAY_OCC,YEAR_OCC,ID,WEEKDAY_OCC
DATE_OCC,,,,,,,,,,,,,,,,,,,,,
2010-01-01,900,21,Topanga,2175,2,354,THEFT OF IDENTITY,0377,48,F,...,34.1720,-118.6084,7,31,2019,1,1,2010,1,Friday
2010-01-01,800,11,Northeast,1112,2,354,THEFT OF IDENTITY,NaN,55,F,...,34.1259,-118.2244,7,21,2011,1,1,2010,2,Friday
2010-01-01,10,20,Olympic,2002,2,354,THEFT OF IDENTITY,0100 1822 0917,42,M,...,34.0783,-118.3113,7,21,2014,1,1,2010,4,Friday


In [4]:
df_top10_crime.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1816075 entries, 2010-01-01 to 2023-08-28
Data columns (total 29 columns):
 #   Column          Dtype  
---  ------          -----  
 0   TIME_OCC        int64  
 1   AREA            int64  
 2   AREA_NAME       object 
 3   RPT_DIST_NO     int64  
 4   PART_1-2        int64  
 5   CRM_CD          int64  
 6   CRM_CD_DESC     object 
 7   MOCODES         object 
 8   VICT_AGE        int64  
 9   VICT_SEX        object 
 10  VICT_DESCENT    object 
 11  PREMIS_CD       float64
 12  PREMIS_DESC     object 
 13  WEAPON_USED_CD  float64
 14  WEAPON_DESC     object 
 15  STATUS          object 
 16  STATUS_DESC     object 
 17  CRM_CD_1        float64
 18  LOCATION        object 
 19  latitude        float64
 20  longitude       float64
 21  MTH_RPTD        int64  
 22  DAY_RPTD        int64  
 23  YEAR_RPTD       int64  
 24  MTH_OCC         int64  
 25  DAY_OCC         int64  
 26  YEAR_OCC        int64  
 27  ID              int64  
 28  WEEKD

In [5]:
df_top10_crime.describe()

,TIME_OCC,AREA,RPT_DIST_NO,PART_1-2,CRM_CD,VICT_AGE,PREMIS_CD,WEAPON_USED_CD,CRM_CD_1,latitude,longitude,MTH_RPTD,DAY_RPTD,YEAR_RPTD,MTH_OCC,DAY_OCC,YEAR_OCC,ID
count,1.816075e+06,1.816075e+06,1.816075e+06,1.816075e+06,1.816075e+06,1.816075e+06,1.816051e+06,576514.000000,1.816075e+06,1.816075e+06,1.816075e+06,1.816075e+06,1.816075e+06,1.816075e+06,1.816075e+06,1.816075e+06,1.816075e+06,1.816075e+06
mean,1.356450e+03,1.093247e+01,1.139489e+03,1.403777e+00,4.640763e+02,3.104446e+01,2.962443e+02,373.920937,4.640622e+02,3.404857e+01,-1.182588e+02,6.435592e+00,1.571329e+01,2.016514e+03,6.423014e+00,1.545008e+01,2.016469e+03,1.426282e+06
std,6.583391e+02,5.975726e+00,5.974912e+02,4.906539e-01,1.502626e+02,2.126195e+01,2.141138e+02,97.766544,1.502647e+02,9.863565e-01,3.403985e+00,3.407404e+00,8.812746e+00,3.940847e+00,3.419783e+00,8.930076e+00,3.942006e+00,8.260731e+05
min,1.000000e+00,1.000000e+00,1.000000e+02,1.000000e+00,2.300000e+02,-1.200000e+01,1.010000e+02,101.000000,-1.000000e+00,0.000000e+00,-1.188279e+02,1.000000e+00,1.000000e+00,2.010000e+03,1.000000e+00,1.000000e+00,2.010000e+03,1.000000e+00
25%,9.000000e+02,6.000000e+00,6.360000e+02,1.000000e+00,3.300000e+02,1.900000e+01,1.010000e+02,400.000000,3.300000e+02,3.401090e+01,-1.184334e+02,4.000000e+00,8.000000e+00,2.013000e+03,3.000000e+00,8.000000e+00,2.013000e+03,7.096875e+05
50%,1.430000e+03,1.100000e+01,1.169000e+03,1.000000e+00,4.400000e+02,3.100000e+01,2.030000e+02,400.000000,4.400000e+02,3.406170e+01,-1.183267e+02,6.000000e+00,1.600000e+01,2.017000e+03,6.000000e+00,1.500000e+01,2.017000e+03,1.418445e+06
75%,1.900000e+03,1.600000e+01,1.641000e+03,2.000000e+00,6.240000e+02,4.600000e+01,5.010000e+02,400.000000,6.240000e+02,3.417210e+01,-1.182761e+02,9.000000e+00,2.300000e+01,2.020000e+03,9.000000e+00,2.300000e+01,2.020000e+03,2.147369e+06
max,2.359000e+03,2.100000e+01,2.199000e+03,2.000000e+00,7.400000e+02,1.200000e+02,9.730000e+02,516.000000,9.990000e+02,3.479070e+01,0.000000e+00,1.200000e+01,3.100000e+01,2.023000e+03,1.200000e+01,3.100000e+01,2.023000e+03,2.865898e+06


In [6]:
df_top10_crime.shape

(1816075, 29)

### 2.Identify and convert categorical variables into dummy/indicator features.

In [7]:
# Identify categorical columns to convert to dummy variables
categorical_cols = df_top10_crime.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()
print(categorical_cols)

['AREA_NAME', 'CRM_CD_DESC', 'MOCODES', 'VICT_SEX', 'VICT_DESCENT', 'PREMIS_DESC', 'WEAPON_DESC', 'STATUS', 'STATUS_DESC', 'LOCATION', 'WEEKDAY_OCC']


<p>After re-looking the columns list.I decided to remove the unnessary columns before encoding ('MOCODES','VICT_DESCENT', 'PREMIS_DESC', 'WEAPON_DESC', 'STATUS', 'STATUS_DESC')  Since modeling will be focus on  'AREA_NAME', 'CRM_CD_DESC','WEEKDAY_OCC'.</p> </br>
<p>ID column will be also removed and reinstate after cleaning duplicate.</p>

In [8]:
columns_to_drop = ['ID','MOCODES','VICT_DESCENT', 'PREMIS_DESC', 'WEAPON_DESC', 'STATUS', 'STATUS_DESC','LOCATION']
df_top10_crime.drop(columns=columns_to_drop, axis=1, inplace=True)

In [9]:
categorical_cols = df_top10_crime.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()
print(categorical_cols)

['AREA_NAME', 'CRM_CD_DESC', 'VICT_SEX', 'WEEKDAY_OCC']


Categorical columns are converted to string type for consistency before applying one-hot encoding.

In [10]:
# Convert all categorical columns to strings to ensure consistency
df_top10_crime[categorical_cols] = df_top10_crime[categorical_cols].astype(str)


In [11]:
df_top10_crime.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1816075 entries, 2010-01-01 to 2023-08-28
Data columns (total 21 columns):
 #   Column          Dtype  
---  ------          -----  
 0   TIME_OCC        int64  
 1   AREA            int64  
 2   AREA_NAME       object 
 3   RPT_DIST_NO     int64  
 4   PART_1-2        int64  
 5   CRM_CD          int64  
 6   CRM_CD_DESC     object 
 7   VICT_AGE        int64  
 8   VICT_SEX        object 
 9   PREMIS_CD       float64
 10  WEAPON_USED_CD  float64
 11  CRM_CD_1        float64
 12  latitude        float64
 13  longitude       float64
 14  MTH_RPTD        int64  
 15  DAY_RPTD        int64  
 16  YEAR_RPTD       int64  
 17  MTH_OCC         int64  
 18  DAY_OCC         int64  
 19  YEAR_OCC        int64  
 20  WEEKDAY_OCC     object 
dtypes: float64(5), int64(12), object(4)
memory usage: 304.8+ MB


<p>Check if there is any duplicate records after removing raw data unique IDs .</p>

In [12]:
#Remove dupliate and keep as new cleaned dataframes
df_top10_crime= df_top10_crime.drop_duplicates(keep='last')

In [13]:
df_top10_crime.shape

(1813555, 21)

<p>Add ID sort by index</p>

In [14]:
# Sort DataFrame by date column
df_top10_crime = df_top10_crime.sort_values(by='DATE_OCC')

# Reset index if needed
df_top10_crime = df_top10_crime.reset_index(drop=True)

# Add index+1 as ID column
df_top10_crime['ID'] = df_top10_crime.index + 1

One-hot encoding is applied, transforming categorical variables into a set of binary columns, each representing a category's presence or absence.

In [15]:
# Apply one-hot encoding to categorical variables
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
df_dummies = pd.DataFrame(ohe.fit_transform(df_top10_crime[categorical_cols]), columns=ohe.get_feature_names(categorical_cols))

In [16]:
df_dummies.head(5)

,AREA_NAME_77th Street,AREA_NAME_Central,AREA_NAME_Devonshire,AREA_NAME_Foothill,AREA_NAME_Harbor,AREA_NAME_Hollenbeck,AREA_NAME_Hollywood,AREA_NAME_Mission,AREA_NAME_N Hollywood,AREA_NAME_Newton,...,VICT_SEX_N,VICT_SEX_Unknown,VICT_SEX_X,WEEKDAY_OCC_Friday,WEEKDAY_OCC_Monday,WEEKDAY_OCC_Saturday,WEEKDAY_OCC_Sunday,WEEKDAY_OCC_Thursday,WEEKDAY_OCC_Tuesday,WEEKDAY_OCC_Wednesday
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
# Drop the original categorical columns and concatenate the new dummy variables
df_top10_crime = df_top10_crime.drop(categorical_cols, axis=1)
df_top10_crime = pd.concat([df_top10_crime, df_dummies], axis=1)

In [18]:
df_top10_crime.shape

(1813555, 63)

In [19]:
df_top10_crime.head(3)

,TIME_OCC,AREA,RPT_DIST_NO,PART_1-2,CRM_CD,VICT_AGE,PREMIS_CD,WEAPON_USED_CD,CRM_CD_1,latitude,...,VICT_SEX_N,VICT_SEX_Unknown,VICT_SEX_X,WEEKDAY_OCC_Friday,WEEKDAY_OCC_Monday,WEEKDAY_OCC_Saturday,WEEKDAY_OCC_Sunday,WEEKDAY_OCC_Thursday,WEEKDAY_OCC_Tuesday,WEEKDAY_OCC_Wednesday
0,900,21,2175,2,354,48,502.0,NaN,354.0,34.1720,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,600,11,1134,2,354,70,501.0,NaN,354.0,34.1145,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,900,1,157,2,354,33,504.0,NaN,354.0,34.0435,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


### 3.Standardize the magnitude of numeric features using a scaler.

In [20]:
# Identify numerical columns for scaling
numerical_cols = df_top10_crime.select_dtypes(include=['int64', 'float64']).columns.tolist()
print (numerical_cols)

['TIME_OCC', 'AREA', 'RPT_DIST_NO', 'PART_1-2', 'CRM_CD', 'VICT_AGE', 'PREMIS_CD', 'WEAPON_USED_CD', 'CRM_CD_1', 'latitude', 'longitude', 'MTH_RPTD', 'DAY_RPTD', 'YEAR_RPTD', 'MTH_OCC', 'DAY_OCC', 'YEAR_OCC', 'ID', 'AREA_NAME_77th Street', 'AREA_NAME_Central', 'AREA_NAME_Devonshire', 'AREA_NAME_Foothill', 'AREA_NAME_Harbor', 'AREA_NAME_Hollenbeck', 'AREA_NAME_Hollywood', 'AREA_NAME_Mission', 'AREA_NAME_N Hollywood', 'AREA_NAME_Newton', 'AREA_NAME_Northeast', 'AREA_NAME_Olympic', 'AREA_NAME_Pacific', 'AREA_NAME_Rampart', 'AREA_NAME_Southeast', 'AREA_NAME_Southwest', 'AREA_NAME_Topanga', 'AREA_NAME_Van Nuys', 'AREA_NAME_West LA', 'AREA_NAME_West Valley', 'AREA_NAME_Wilshire', 'CRM_CD_DESC_ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT', 'CRM_CD_DESC_BATTERY - SIMPLE ASSAULT', 'CRM_CD_DESC_BURGLARY', 'CRM_CD_DESC_BURGLARY FROM VEHICLE', 'CRM_CD_DESC_INTIMATE PARTNER - SIMPLE ASSAULT', 'CRM_CD_DESC_THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER)', 'CRM_CD_DESC_THEFT OF IDENTITY', 'CRM_CD_D

Numerical columns are standardized using StandardScaler to ensure uniformity in their magnitude, which is crucial for many machine learning algorithms.

In [21]:
# Apply standard scaling to numerical features
scaler = StandardScaler()
df_top10_crime[numerical_cols] = scaler.fit_transform(df_top10_crime[numerical_cols])

In [22]:
# Combine with numeric data
df_numeric = df_top10_crime[numerical_cols]
df_processed = pd.concat([df_numeric, df_dummies], axis=1)

In [23]:
df_numeric.head(3)

,TIME_OCC,AREA,RPT_DIST_NO,PART_1-2,CRM_CD,VICT_AGE,PREMIS_CD,WEAPON_USED_CD,CRM_CD_1,latitude,...,VICT_SEX_N,VICT_SEX_Unknown,VICT_SEX_X,WEEKDAY_OCC_Friday,WEEKDAY_OCC_Monday,WEEKDAY_OCC_Saturday,WEEKDAY_OCC_Sunday,WEEKDAY_OCC_Thursday,WEEKDAY_OCC_Tuesday,WEEKDAY_OCC_Wednesday
0,-0.693489,1.684796,1.733157,1.215008,-0.733384,0.796716,0.960800,NaN,-0.73328,0.125106,...,-0.002572,-0.433481,-0.164708,2.344714,-0.40622,-0.413561,-0.40285,-0.404006,-0.400699,-0.40366
1,-1.149252,0.011354,-0.009133,1.215008,-0.733384,1.831364,0.956129,NaN,-0.73328,0.066831,...,-0.002572,-0.433481,-0.164708,2.344714,-0.40622,-0.413561,-0.40285,-0.404006,-0.400699,-0.40366
2,-0.693489,-1.662089,-1.644309,1.215008,-0.733384,0.091274,0.970141,NaN,-0.73328,-0.005126,...,-0.002572,-0.433481,-0.164708,2.344714,-0.40622,-0.413561,-0.40285,-0.404006,-0.400699,-0.40366


In [24]:
df_processed.head(3)

,TIME_OCC,AREA,RPT_DIST_NO,PART_1-2,CRM_CD,VICT_AGE,PREMIS_CD,WEAPON_USED_CD,CRM_CD_1,latitude,...,VICT_SEX_N,VICT_SEX_Unknown,VICT_SEX_X,WEEKDAY_OCC_Friday,WEEKDAY_OCC_Monday,WEEKDAY_OCC_Saturday,WEEKDAY_OCC_Sunday,WEEKDAY_OCC_Thursday,WEEKDAY_OCC_Tuesday,WEEKDAY_OCC_Wednesday
0,-0.693489,1.684796,1.733157,1.215008,-0.733384,0.796716,0.960800,NaN,-0.73328,0.125106,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-1.149252,0.011354,-0.009133,1.215008,-0.733384,1.831364,0.956129,NaN,-0.73328,0.066831,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.693489,-1.662089,-1.644309,1.215008,-0.733384,0.091274,0.970141,NaN,-0.73328,-0.005126,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
# Apply standard scaling to the numerical features
scaler = StandardScaler()
df_processed[numerical_cols] = scaler.fit_transform(df_processed[numerical_cols])

In [26]:
df_processed.head(3)

,TIME_OCC,AREA,RPT_DIST_NO,PART_1-2,CRM_CD,VICT_AGE,PREMIS_CD,WEAPON_USED_CD,CRM_CD_1,latitude,...,VICT_SEX_N,VICT_SEX_Unknown,VICT_SEX_X,WEEKDAY_OCC_Friday,WEEKDAY_OCC_Monday,WEEKDAY_OCC_Saturday,WEEKDAY_OCC_Sunday,WEEKDAY_OCC_Thursday,WEEKDAY_OCC_Tuesday,WEEKDAY_OCC_Wednesday
0,-0.693489,1.684796,1.733157,1.215008,-0.733384,0.796716,0.960800,NaN,-0.73328,0.125106,...,-0.002572,-0.433481,-0.164708,2.344714,-0.40622,-0.413561,-0.40285,-0.404006,-0.400699,-0.40366
1,-1.149252,0.011354,-0.009133,1.215008,-0.733384,1.831364,0.956129,NaN,-0.73328,0.066831,...,-0.002572,-0.433481,-0.164708,2.344714,-0.40622,-0.413561,-0.40285,-0.404006,-0.400699,-0.40366
2,-0.693489,-1.662089,-1.644309,1.215008,-0.733384,0.091274,0.970141,NaN,-0.73328,-0.005126,...,-0.002572,-0.433481,-0.164708,2.344714,-0.40622,-0.413561,-0.40285,-0.404006,-0.400699,-0.40366


In [27]:
df_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1813555 entries, 0 to 1813554
Columns: 108 entries, TIME_OCC to WEEKDAY_OCC_Wednesday
dtypes: float64(108)
memory usage: 1.5 GB


The target variables are going to be removed from the features set.the remaining data are treated as features. The dataset is split into feature (X) and target (y) sets.

In [28]:
target_var_cols = ['VICT_AGE', 'latitude', 'longitude', 'MTH_RPTD', 'DAY_RPTD', 'YEAR_RPTD', 'MTH_OCC', 'DAY_OCC', 'YEAR_OCC']
X = df_top10_crime.drop(columns=target_var_cols, axis=1)
y = df_top10_crime[target_var_cols]

In [29]:
print (X)

         TIME_OCC      AREA  RPT_DIST_NO  PART_1-2    CRM_CD  PREMIS_CD  \
0       -0.693489  1.684796     1.733157  1.215008 -0.733384   0.960800   
1       -1.149252  0.011354    -0.009133  1.215008 -0.733384   0.956129   
2       -0.693489 -1.662089    -1.644309  1.215008 -0.733384   0.970141   
3        0.521881  0.848075     0.906364  1.215008  1.064148  -0.403027   
4        0.544669 -0.992712    -1.063545  1.215008  1.064148  -0.912127   
...           ...       ...          ...       ...       ...        ...   
1813550 -1.734148  0.513386     0.482926 -0.823040 -1.026316   0.960800   
1813551  0.582649 -1.160056    -1.118777 -0.823040 -1.558918  -0.912127   
1813552 -0.951754  0.011354    -0.064364 -0.823040 -0.893165  -0.879432   
1813553 -1.855685  0.513386     0.504683  1.215008  1.064148  -0.417039   
1813554 -2.059260  0.178698     0.159907 -0.823040  0.305190  -0.912127   

         WEAPON_USED_CD  CRM_CD_1        ID  AREA_NAME_77th Street  ...  \
0                   NaN 

In [30]:
print (y)

         VICT_AGE  latitude  longitude  MTH_RPTD  DAY_RPTD  YEAR_RPTD  \
0        0.796716  0.125106  -0.102694   0.16562  1.734663   0.630897   
1        1.831364  0.066831   0.006198   1.33954  0.826873  -1.652892   
2        0.091274 -0.005126   0.004700  -0.12786 -0.534811  -0.637875   
3       -0.049815  0.177097  -0.040436  -1.59526 -1.669548  -1.652892   
4       -0.284962 -0.206401  -0.013301  -1.59526 -1.669548  -1.652892   
...           ...       ...        ...       ...       ...        ...   
1813550 -0.002785 -0.022355  -0.046897   0.45910  1.394242   1.645915   
1813551 -1.460699 -0.017693   0.011308   0.45910  1.394242   1.645915   
1813552  0.373451  0.070074  -0.010012   0.45910  1.394242   1.645915   
1813553 -0.143873 -0.070292  -0.061052   0.45910  1.394242   1.645915   
1813554 -1.460699 -0.065326  -0.012215   0.45910  1.394242   1.645915   

          MTH_OCC   DAY_OCC  YEAR_OCC  
0       -1.585802 -1.618106 -1.640931  
1       -1.585802 -1.618106 -1.640931  
2  

### 4.Split the dataset into training and testing datasets.

The dataset is further split into training and testing sets, following the common practice of holding out a portion of the data for model evaluation.

In [31]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [32]:
# Output the shapes of the resulting dataframes
(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

((1450844, 54), (362711, 54), (1450844, 9), (362711, 9))

### 5.Modeling

In [34]:
#Calculate the mean of `y_train`
train_mean = y_train.mean()
train_mean

VICT_AGE     0.000503
latitude     0.000587
longitude   -0.000598
MTH_RPTD    -0.000415
DAY_RPTD    -0.001293
YEAR_RPTD   -0.000273
MTH_OCC     -0.000516
DAY_OCC     -0.000967
YEAR_OCC    -0.000265
dtype: float64

In [35]:
#Calculate the medium of `y_train`
X_defaults_median = X_train.median()
X_defaults_median

TIME_OCC                                                               0.111694
AREA                                                                   0.011354
RPT_DIST_NO                                                            0.049445
PART_1-2                                                              -0.823040
CRM_CD                                                                -0.160837
PREMIS_CD                                                             -0.435721
WEAPON_USED_CD                                                         0.264450
CRM_CD_1                                                              -0.160741
ID                                                                    -0.000114
AREA_NAME_77th Street                                                 -0.268962
AREA_NAME_Central                                                     -0.230213
AREA_NAME_Devonshire                                                  -0.216273
AREA_NAME_Foothill                      

##### Impute NaN Data

In [54]:
# handling NaN values
# Replace 'mean' with 'median', 'most_frequent', or a constant value if needed

imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
X_train = imputer.fit_transform(X_train)
y_train = imputer.fit_transform(y_train)

In [55]:
# Check for NaN values
print("NaN values in X_test:", X_test.isna().sum().sum())

# Replace NaN values with the mean (you can choose other methods)
X_test = X_test.fillna(X_test.mean())

# Check for infinity values
print("Infinity values in X_test:", np.isinf(X_test).sum().sum())

# Replace infinity values
X_test = X_test.replace([np.inf, -np.inf], np.nan).fillna(X_test.mean())

NaN values in X_test: 247492
Infinity values in X_test: 0


In [57]:
# Normalize or Scale your data
scaler = StandardScaler()
X_test_scaled = scaler.fit_transform(X_test)
X_test_scaled

array([[ 1.32850326, -0.32274956, -0.34163968, ...,  2.47591504,
        -0.40023136, -0.40384952],
       [-0.16208818,  0.34710899,  0.37518183, ..., -0.40389108,
        -0.40023136,  2.47616982],
       [ 1.1764021 ,  0.68203826,  0.70009625, ..., -0.40389108,
        -0.40023136, -0.40384952],
       ...,
       [ 0.97867058,  0.8495029 ,  0.84078085, ..., -0.40389108,
        -0.40023136, -0.40384952],
       [ 0.87219976,  1.01696754,  0.99821361, ..., -0.40389108,
        -0.40023136,  2.47616982],
       [-0.08603759, -0.49021419, -0.42203087, ...,  2.47591504,
        -0.40023136, -0.40384952]])

#### Basic Dummy Regressor

In [37]:
#Fit the dummy regressor on the training data
#Hint, call its `.fit()` method with `X_train` and `y_train` as arguments
#Then print the object's `constant_` attribute and verify it's the same as the mean above
dumb_reg = DummyRegressor(strategy='mean')
dumb_reg.fit(X_train, y_train)
dumb_reg.constant_

array([[ 0.00050284,  0.00058684, -0.0005982 , -0.00041537, -0.00129335,
        -0.00027281, -0.00051641, -0.00096734, -0.00026528]])

##### Metrics on dummy regressor

###### R2 Square

In [38]:
def r_squared(y, ypred):
    """R-squared score.
    
    Calculate the R-squared, or coefficient of determination, of the input.
    
    Arguments:
    y -- the observed values
    ypred -- the predicted values
    """
    ybar = np.sum(y) / len(y) #yes, we could use np.mean(y)
    sum_sq_tot = np.sum((y - ybar)**2) #total sum of squares error
    sum_sq_res = np.sum((y - ypred)**2) #residual sum of squares error
    R2 = 1.0 - sum_sq_res / sum_sq_tot
    return R2

Remember the sklearn dummy regressor?

In [39]:
y_train_pred = dumb_reg.predict(X_train)
y_train_pred[:5]

array([[ 0.00050284,  0.00058684, -0.0005982 , -0.00041537, -0.00129335,
        -0.00027281, -0.00051641, -0.00096734, -0.00026528],
       [ 0.00050284,  0.00058684, -0.0005982 , -0.00041537, -0.00129335,
        -0.00027281, -0.00051641, -0.00096734, -0.00026528],
       [ 0.00050284,  0.00058684, -0.0005982 , -0.00041537, -0.00129335,
        -0.00027281, -0.00051641, -0.00096734, -0.00026528],
       [ 0.00050284,  0.00058684, -0.0005982 , -0.00041537, -0.00129335,
        -0.00027281, -0.00051641, -0.00096734, -0.00026528],
       [ 0.00050284,  0.00058684, -0.0005982 , -0.00041537, -0.00129335,
        -0.00027281, -0.00051641, -0.00096734, -0.00026528]])

In [40]:
r_squared(y_train, y_train_pred)

8.658867610589205e-06

In [41]:
y_test_pred = dumb_reg.predict(X_test)
r_squared(y_test, y_test_pred)

VICT_AGE    -0.000006
latitude    -0.000008
longitude   -0.000008
MTH_RPTD    -0.000004
DAY_RPTD    -0.000042
YEAR_RPTD   -0.000002
MTH_OCC     -0.000007
DAY_OCC     -0.000023
YEAR_OCC    -0.000002
dtype: float64

###### Mean Square Error

In [42]:
def mse(y, ypred):
    """Mean square error.
    
    Calculate the mean square error of the arguments

    Arguments:
    y -- the observed values
    ypred -- the predicted values
    """
    sq_error = (y - ypred)**2
    mse = np.mean(sq_error)
    return mse

In [43]:
mse_train = mse(y_train, y_train_pred)
mse_train

0.9956148168121295

In [44]:
mse_test = mse(y_test, y_test_pred)
mse_test

/Users/thantthiri/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3417: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


VICT_AGE     0.998649
latitude     1.081431
longitude    1.082467
MTH_RPTD     0.997982
DAY_RPTD     0.998912
YEAR_RPTD    1.001225
MTH_OCC      0.998607
DAY_OCC      0.997824
YEAR_OCC     1.000790
dtype: float64

###### Mean Absolute Error

In [45]:
def mae(y, ypred):
    """Mean absolute error.
    
    Calculate the mean absolute error of the arguments

    Arguments:
    y -- the observed values
    ypred -- the predicted values
    """
    abs_error = np.abs(y - ypred)
    mae = np.mean(abs_error)
    return mae

In [46]:
mae(y_train, y_train_pred)

0.6855192356899321

In [47]:
mae(y_test, y_test_pred)

VICT_AGE     0.807932
latitude     0.121657
longitude    0.062004
MTH_RPTD     0.862139
DAY_RPTD     0.864846
YEAR_RPTD    0.862415
MTH_OCC      0.862872
DAY_OCC      0.864286
YEAR_OCC     0.862204
dtype: float64

So here, you get a slightly better MSE on the test set than you did on the train set. And what does a squared error mean anyway? To convert this back to our measurement space, we often take the square root, to form the root mean square error thus:

In [48]:
# np.sqrt([mse(y_train, y_train_pred), mse(y_test, y_test_pred)])

# Calculate square root of MSE
rmse_train = np.sqrt(mse_train)
rmse_test = np.sqrt(mse_test)

# Now you have the RMSE for both the training and test sets
print("Training RMSE:", rmse_train)
print("Test RMSE:", rmse_test)

Training RMSE: 0.9978049993922307
Test RMSE: VICT_AGE     0.999324
latitude     1.039919
longitude    1.040417
MTH_RPTD     0.998991
DAY_RPTD     0.999456
YEAR_RPTD    1.000612
MTH_OCC      0.999303
DAY_OCC      0.998912
YEAR_OCC     1.000395
dtype: float64


##### sklearn metrics 

Functions are good, but you don't want to have to define functions every time we want to assess performance. sklearn.metrics provides many commonly used metrics, included the ones above.

In [49]:
r2_score(y_train, y_train_pred), r2_score(y_test, y_test_pred)

(-2.6990755309776028e-14, -1.138683803988035e-05)

In [50]:
mean_absolute_error(y_train, y_train_pred), mean_absolute_error(y_test, y_test_pred)

(0.6855192356895663, 0.6855950399149299)

In [51]:
mean_squared_error(y_train, y_train_pred), mean_squared_error(y_test, y_test_pred)

(0.9956148168121545, 1.0175430369592369)

### Initial Modeling

##### Linear Regression

In [58]:
# Train Linear Regression model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

LinearRegression()

In [59]:
# Making predictions
y_pred = lr_model.predict(X_test)

##### Random Forest

In [ ]:
# Train Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

##### Cross Validation

In [ ]:
# Perform cross-validation
lr_cv_scores = cross_val_score(lr_model, X, y, cv=5, scoring='neg_mean_squared_error')
rf_cv_scores = cross_val_score(rf_model, X, y, cv=5, scoring='neg_mean_squared_error')

In [ ]:
 # Predictions for test set
lr_predictions = lr_model.predict(X_test)
rf_predictions = rf_model.predict(X_test)    


In [ ]:
# Evaluation
lr_mse = mean_squared_error(y_test, lr_predictions)
rf_mse = mean_squared_error(y_test, rf_predictions)
    
# Mean Cross-validation scores
lr_cv_mse_mean = np.mean(lr_cv_scores)
rf_cv_mse_mean = np.mean(rf_cv_scores)

In [ ]:
# Print results
results = {
        "Linear Regression Test MSE": lr_mse,
        "Random Forest Test MSE": rf_mse,
        "Linear Regression CV MSE Mean": lr_cv_mse_mean,
        "Random Forest CV MSE Mean": rf_cv_mse_mean,
    }
    results